# Monitoring Locations Field Length Analyzer

## Description

This script was created using the Google Colab Jupyter Notebook web service

This script downloads all monitoring location records from the USGS API endpoint and analyzes the maximum string length of each field across the entire dataset. The purpose is to determine appropriate maximum lengths for each database field to inform schema design and optimize storage.

## How it works
1. **Data Download:**
   - The script uses the USGS API endpoint `https://api.waterdata.usgs.gov/ogcapi/v0/collections/monitoring-locations/items` to retrieve monitoring location data.
   - It downloads data in chunks (pages) of up to 1000 records at a time using `limit` and `offset` parameters.
   - Pagination continues until all available records (about 41,000) are retrieved or an HTTP error occurs indicating no more data.

2. **Data Extraction:**
   - From each downloaded feature, the script extracts the `properties` dictionary, which contains all the fields for that record.

3. **Data Analysis:**
   - Converts the list of properties dictionaries into a Pandas DataFrame.
   - For each field (column), calculates the maximum string length found across all records.
   - Prints out the maximum lengths in descending order.

## Usage
- Run the script in an environment with Python and required packages (`requests`, `pandas`) installed.
- The resulting max field lengths help in setting precise string length limits for database schema fields, improving data integrity and storage efficiency.

## Notes
- The lengths reflect the current dataset snapshot and may not account for future data variations.
- Adding a small buffer to max lengths is recommended for schema design.



In [ ]:
import requests
import time
import json

BASE_URL = "https://api.waterdata.usgs.gov/ogcapi/v0/collections/monitoring-locations/items"
LIMIT = 1000
offset = 0
all_features = []

while True:
    params = {
        "limit": LIMIT,
        "offset": offset,
        "f": "json"
    }

    try:
        response = requests.get(BASE_URL, params=params)
        response.raise_for_status()
    except requests.exceptions.HTTPError as e:
        print(f"HTTP error at offset {offset}: {e}")
        break

    data = response.json()
    features = data.get("features", [])

    if offset == 0:
        total_matched = data.get("numberMatched") or data.get("totalFeatures")
        if total_matched:
            print(f"Total features reported by API: {total_matched}")

    if not features:
        print("No more features returned, stopping.")
        break

    all_features.extend(features)
    print(f"Downloaded {len(features)} features, total so far: {len(all_features)}")

    offset += LIMIT

    # Stop if we've downloaded all or more than total reported (if known)
    if total_matched and offset >= total_matched:
        print("Reached total reported count; stopping.")
        break

    time.sleep(0.1)

print(f"Total features downloaded: {len(all_features)}")

with open("monitoring_locations_all.json", "w") as f:
    json.dump(all_features, f, indent=2)


In [ ]:
import json
import pandas as pd

# Load saved data
with open("monitoring_locations_all.json") as f:
    data = json.load(f)

# Extract 'properties' dicts from each feature
properties_list = [feature['properties'] for feature in data]

# Convert to DataFrame for easy analysis
df = pd.DataFrame(properties_list)

# Calculate max string length per field
max_lengths = {}

for col in df.columns:
    # Convert all to string, then calculate length
    max_len = df[col].astype(str).map(len).max()
    max_lengths[col] = max_len

# Show the max length per field, sorted by length descending
max_lengths_sorted = dict(sorted(max_lengths.items(), key=lambda item: item[1], reverse=True))
for field, length in max_lengths_sorted.items():
    print(f"{field}: {length}")
